# Analysis Notebook
1. Setup
2. Set up directory
3. Load logging file
4. Reverse-engineer dictionary
5. Plot
5a) Loss
5b) Train Accuracy
5c) Test Accuracy

# 1 - Setup

In [1]:
import itertools
import copy
import warnings
import json
import numpy as np
import os
import math
import torch
%load_ext autoreload
%autoreload 2

# 2- Set up directory

In [4]:
path = os.getcwd()
log_path = path + '/../../../data/models/convai2/'
folder = 'profilememory'
session_directory = os.path.join(log_path, folder)

# 3 - Load Data

In [3]:
#results = {}
#loggings = 

file = "logging/log.txt"

with open ((log_path + folder + "/" + file), 'rt') as in_file:  # Open file lorem.txt for reading of text data.
    log = in_file.read()# Read the entire file into a variable named contents.
    print(log) # Print contents.

[ optional arguments: ] 
[  task: convai2:self ]
[  pytorch_teacher_task: None ]
[  pytorch_teacher_dataset: None ]
[  download_path: /home/team_roboy/ParlAI/downloads ]
[  datatype: train ]
[  image_mode: raw ]
[  numthreads: 1 ]
[  hide_labels: False ]
[  batchsize: 64 ]
[  batch_sort: True ]
[  context_length: -1 ]
[  include_labels: True ]
[  pytorch_datafile:  ]
[  numworkers: 4 ]
[  pytorch_preprocess: False ]
[  batch_sort_cache: none ]
[  batch_length_range: 5 ]
[  datapath: /home/team_roboy/ParlAI/data ]
[  model: projects.personachat.persona_seq2seq:PersonachatSeqseqAgentSplit ]
[  model_file: /tmp/roboy_profilemem ]
[  dict_class: None ]
[  evaltask: None ]
[  display_examples: False ]
[  num_epochs: -1 ]
[  max_train_time: -1 ]
[  validation_every_n_secs: 180.0 ]
[  save_every_n_secs: 1800.0 ]
[  save_after_valid: False ]
[  validation_every_n_epochs: -1 ]
[  validation_max_exs: -1 ]
[  validation_patience: 20 ]
[  validation_metric: f1 ]
[  validation_metric_mode: max ]
[ 

# 4 - Create Dictionary

In [ ]:
# for profilememory
# reverse engineer logged shell output from string to dictionary
log_dict = dict()
time_between_validations = 180
time_for_validations = 150

# needed to extract relevant entreis form string
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""


with open ((path + folder + "/" + file), 'rt') as fp: 
    line = fp.readline()
    cnt = 1
    dict_cnt = 1
    val_time = time_between_validations
    while line:
        teststring = line.strip()
        
        if (teststring[0:7] == '[ time:') == True:
        # filter for relevant lines
        # [ time:15s total_exs:6528 total_eps:0.05 ] {'exs': 6528, 'accuracy': 0, 'f1': 0.1009}
        # [ time:310s total_exs:84078 total_eps:0.64 ] {'exs': 6784, 'accuracy': 0, 'f1': 0.2383, 'loss': 3.982, 'ppl': 53.6} 
        
        # extract relevant entries form string
            #first block []
            time = float(find_between( teststring, "time:", "s " ))   
            total_exs = float(find_between( teststring, "total_exs:", " " ))
            total_eps = float(find_between( teststring, "total_eps:", " ]" ))

            # second block {}            
            exs = float(find_between( teststring, "total_exs:", " " ))
            accuracy = float(find_between( teststring, "'accuracy': ", ", " ))
            # f1
            try:
                train_f1 = float(find_between( teststring, "'f1': ", ", " ))
            except:
                train_f1 = float(find_between( teststring, "'f1': ", "}" ))
            # loss  
            try:
                train_loss = float(find_between( teststring, "'loss': ", ", " ))
            except: 
                train_loss = None
            # ppl
            try:
                train_ppl = float(find_between( teststring, "'ppl': ", "}" ))
            except:
                train_ppl = None                
            
            #save to dictionary
            log_dict[dict_cnt] = {'time': time, 'total_exs': total_exs, 'total_eps': total_eps,
                                  'exs': exs, 
                                  'train_accuracy': accuracy, 'val_accuracy': None, 
                                  'train_f1': train_f1, 'val_f1:': None,
                                  'train_loss': train_loss, 'val_loss': None, 
                                  'hits@1': None, 'hits@5': None, 'hits@10': None, 'hits@100': None,
                                  'train_ppl': train_ppl}
            dict_cnt += 1
        
        elif (teststring[0:7] == 'valid:{') == True:
        # filter for relevant log validations every x seconds
        # valid:{'exs': 7801, 'accuracy': 0, 'f1': 0.02814, 'hits@1': 0.0663, 'hits@5': 0.29, 'hits@10': 0.547, 'hits@100': 1.0, 'loss': 3.982, 'ppl': 53.6}
            exs = float(find_between( teststring, "'exs': ", ", " ))
            accuracy = float(find_between( teststring, "'accuracy': ", ", " ))
            val_f1 = float(find_between( teststring, "'f1': ", ", " ))
            hits1 = float(find_between( teststring, "'hits@1': ", ", " ))
            hits5 = float(find_between( teststring, "'hits@5': ", ", " ))
            hits10 = float(find_between( teststring, "'hits@10': ", ", " ))
            hits100 = float(find_between( teststring, "'hits@100': ", ", " ))
            val_loss = float(find_between( teststring, "'loss': ", ", " ))
            val_ppl = float(find_between( teststring, "'ppl': ", "}" ))
            
            # save to dictionary
            log_dict[dict_cnt] = {'time': val_time, 'total_exs': None, 'exs': exs, 
                                  'train_accuracy': None, 'val_accuracy': accuracy, 
                                  'train_f1': None, 'val_f1': val_f1, 
                                  'train_loss': None, 'val_loss': val_loss, 
                                  'hits@1': hits1, 'hits@5': hits5, 'hits@10': hits10, 'hits@100': hits100, 
                                  'val_ppl': val_ppl}
            
            # advance validation time for next iteration and dict
            val_time += time_between_validations+time_for_validations
            dict_cnt += 1
            
        elif (teststring[0:43] == '[ ran out of patience! stopping training. ]') == True:
            print('Training complete!')

        
        line = fp.readline()
        cnt += 1
# seq2se#print(log_dict)

In [ ]:
# for seq2seq + attention
# reverse engineer logged shell output from string to dictionary
log_dict = dict()
time_between_validations = 180
time_for_validations = 150

# needed to extract relevant entreis form string
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""


with open ((path + folder + "/" + file), 'rt') as fp: 
    line = fp.readline()
    cnt = 1
    dict_cnt = 1
    val_time = time_between_validations
    while line:
        teststring = line.strip()
        
        if (teststring[0:7] == '[ time:') == True:
        # filter for relevant lines
        #[ time:2373s total_exs:432010 total_eps:3.29 ] {'exs': 2816, 'token_acc': 0.3239, 'loss': 3.603, 'ppl': 36.72}        
        # valid:{'exs': 7801, 'accuracy': 0.0006409, 'f1': 0.1331, 'loss': 3.875, 'ppl': 48.19}
        # extract relevant entries form string
            #first block []
            time = float(find_between( teststring, "time:", "s " ))   
            total_exs = float(find_between( teststring, "total_exs:", " " ))
            total_eps = float(find_between( teststring, "total_eps:", " ]" ))

            # second block {}            
            exs = float(find_between( teststring, "total_exs:", " " ))
            #accuracy = float(find_between( teststring, "'accuracy': ", ", " ))
            # token_acc
            try:
                train_f1 = float(find_between( teststring, "'token_acc': ", ", " ))
            except:
                train_f1 = float(find_between( teststring, "'token_acc': ", "}" ))
            # loss  
            try:
                train_loss = float(find_between( teststring, "'loss': ", ", " ))
            except: 
                train_loss = None
            # ppl
            try:
                train_ppl = float(find_between( teststring, "'ppl': ", "}" ))
            except:
                train_ppl = None                
            
            #save to dictionary
            log_dict[dict_cnt] = {'time': time, 'total_exs': total_exs, 'total_eps': total_eps,
                                  'exs': exs, 'val_accuracy': None, 
                                  'train_f1': train_f1, 'val_f1:': None,
                                  'train_loss': train_loss, 'val_loss': None, 
                                  'hits@1': None, 'hits@5': None, 'hits@10': None, 'hits@100': None,
                                  'train_ppl': train_ppl}
            dict_cnt += 1
        
        elif (teststring[0:7] == 'valid:{') == True:
        # filter for relevant log validations every x seconds
        # valid:{'exs': 7801, 'accuracy': 0, 'f1': 0.02814, 'hits@1': 0.0663, 'hits@5': 0.29, 'hits@10': 0.547, 'hits@100': 1.0, 'loss': 3.982, 'ppl': 53.6}
            exs = float(find_between( teststring, "'exs': ", ", " ))
            accuracy = float(find_between( teststring, "'accuracy': ", ", " ))
            val_f1 = float(find_between( teststring, "'f1': ", ", " ))
            #hits1 = float(find_between( teststring, "'hits@1': ", ", " ))
            #hits5 = float(find_between( teststring, "'hits@5': ", ", " ))
            #hits10 = float(find_between( teststring, "'hits@10': ", ", " ))
            #hits100 = float(find_between( teststring, "'hits@100': ", ", " ))
            val_loss = float(find_between( teststring, "'loss': ", ", " ))
            val_ppl = float(find_between( teststring, "'ppl': ", "}" ))
            
            # save to dictionary
            log_dict[dict_cnt] = {'time': val_time, 'total_exs': None, 'exs': exs, 
                                  'train_accuracy': None, 'val_accuracy': accuracy, 
                                  'train_f1': None, 'val_f1': val_f1, 
                                  'train_loss': None, 'val_loss': val_loss,  
                                  'val_ppl': val_ppl}
            
            # advance validation time for next iteration and dict
            val_time += time_between_validations+time_for_validations
            dict_cnt += 1
            
        elif (teststring[0:43] == '[ ran out of patience! stopping training. ]') == True:
            print('Training complete!')

        
        line = fp.readline()
        cnt += 1
#print(log_dict)

# 5 - Plots

In [ ]:
### plot parameters
#fontsizes
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fs_title = 24
fs_ax_labels = 20
fs_tick_labels=18
fs_legend = 20

is_final_plot = True

# 5.1 - LOSS

In [ ]:
# Plot loss
# ParlAI gives loss form validation

fig_trloss=plt.figure(dpi= 80, facecolor='w', edgecolor='k')

plot = plt.subplot(2, 1, 1)
plt.title('Validation Loss',fontsize= fs_title)

for i in range(1,len(log_dict)):
    try:
        val_loss = log_dict[i]['val_loss']
        plt.plot(time, val_loss, marker = 'o', color = 'blue')
    except:
        pass
    time = log_dict[i]['time']
    #try:
    #    epochs = log_dict[i]['total_eps']
    #    train_loss = log_dict[i]['train_loss']
    #    plt.plot(time, train_loss, marker = 'o', color = 'blue')
    #except:
    #    pass


plt.xlabel('Time [s]', fontsize= fs_ax_labels)

plt.gcf().set_size_inches(15, 12)
plt.tick_params(axis='both', which='major', labelsize=fs_tick_labels)
plot.yaxis.grid()
plt.show()

# 5.2 - Perplexity

In [ ]:
# Plot PPL score

fig_ppl=plt.figure(dpi= 80, facecolor='w', edgecolor='k')
plot = fig_ppl.add_subplot(111)

plt.title('Validation PPL', fontsize=fs_title)

for i in range(1,len(log_dict)):
    time = log_dict[i]['time']
    #try:
    #    train_f1 = log_dict[i]['train_ppl']
    #    plt.scatter(time, train_f1, marker = 'o', color = 'blue')
    #except:
    #    pass
    try:
        val_f1 = log_dict[i]['val_ppl']
        plt.scatter(time, val_f1, marker = 'o', color = 'blue')
    except:
        pass
        
plt.xlabel('Time [s]', fontsize = fs_ax_labels)
plt.gcf().set_size_inches(15, 6)
plot.tick_params(axis='both', which='major', labelsize=fs_tick_labels)
plot.yaxis.grid()
plt.show()

# 5.3 - F1

In [ ]:
# Plot F1 score

#fig_f1hits=plt.figure(dpi= 80, facecolor='w', edgecolor='k')
#plot, ax1 = fig_f1hits.add_subplot(111)
fig_f1hits, ax1 = plt.subplots()

#plot = plt.subplot(2, 1, 1)
plt.title('F1', fontsize=fs_title)
epochs = []
ax1.scatter(0, 0, marker = 'o', color = 'white')
for i in range(1,len(log_dict)):
    time = log_dict[i]['time']
    try:
        val_f1 = log_dict[i]['val_f1']
        ax1.scatter(time, val_f1, marker = 'o', color = 'red')
        print(val_f1)
    except:
        pass

plt.xlabel('Time [s]', fontsize = fs_ax_labels)
plt.gcf().set_size_inches(15, 6)
plot.tick_params(axis='both', which='major', labelsize=fs_tick_labels)

ax2 = ax1.twiny()

ax2.scatter(0, 0, marker = 'o', color = 'white')
for i in range(1,len(log_dict)):
    try:
        epoch = log_dict[i]['total_eps']
        train_f1 = log_dict[i]['train_f1']
        ax2.scatter(epoch, train_f1, marker = 'o', color = 'blue')
    except:
        pass

epoch_scale = np.linspace(0,int(epoch),int(epoch)+1)
ax2.xaxis.set_ticks_position('bottom') # set the position of the second x-axis to bottom
ax2.xaxis.set_label_position('bottom') # set the position of the second x-axis to bottom
ax2.spines['bottom'].set_position(('outward', 50))
ax2.set_xlabel('Epochs', fontsize = fs_ax_labels)
ax2.set_xticks(epoch_scale)
#ax2.set_xlim(plt.get_xlim())

blue_patch = mpatches.Patch(color='blue', label='Training Score')
red_patch = mpatches.Patch(color='red', label='Validation Score')
ax1.yaxis.grid()
plt.legend(handles=[blue_patch, red_patch])
plt.hlines(0.15,0,epoch,linewidth=0.5,color='black')
plt.hlines(0.16,0,epoch,linewidth=0.5,color='black')
plt.hlines(0.1709,0,epoch,linewidth=0.5,color='red')
plt.show()

# 5.4 - Hits@X

In [ ]:
# Plot Hits @ X

fig_hitsX=plt.figure(dpi= 80, facecolor='w', edgecolor='k')
plot = fig_hitsX.add_subplot(111)

#plot = plt.subplot(2, 1, 1)
plt.title('Hits@X', fontsize=fs_title)

for i in range(1,len(log_dict)):
    time = log_dict[i]['time']
    #epochs = log_dict[i]['total_eps']

    try:
        hits1 = log_dict[i]['hits@1']
        hits5 = log_dict[i]['hits@5']
        hits10 = log_dict[i]['hits@10']
        hits100 = log_dict[i]['hits@100']
        plt.scatter(time, hits1, marker = 'o', color = 'blue')
        plt.scatter(time, hits5, marker = 'o', color = 'red')
        plt.scatter(time, hits10, marker = 'o', color = 'purple')
        plt.scatter(time, hits100, marker = 'o', color = 'green')
    except:
        pass
        
plt.xlabel('Time [s]', fontsize = fs_ax_labels)
plt.gcf().set_size_inches(15, 6)
plot.tick_params(axis='both', which='major', labelsize=fs_tick_labels)
plot.yaxis.grid()
plt.show()

# 5.5 - Accuracies

In [ ]:
# Plot training accuracy

fig_tr_acc=plt.figure(dpi= 80, facecolor='w', edgecolor='k')
plot = fig_tr_acc.add_subplot(111)

#plot = plt.subplot(2, 1, 1)
plt.title('Training Accuracy', fontsize=fs_title)


for i in range(1,len(log_dict)):
    time = log_dict[i]['time']
    acc = log_dict[i]['train_accuracy']
    if acc != None:
        plt.plot(time, acc, marker = 'o', color = 'blue')

        
plt.xlabel('Time [s]', fontsize = fs_ax_labels)
plt.gcf().set_size_inches(15, 6)
plot.tick_params(axis='both', which='major', labelsize=fs_tick_labels)
plot.yaxis.grid()
plt.show()

In [ ]:
# Plot valdiation accuracy

fig_val_acc=plt.figure(dpi= 80, facecolor='w', edgecolor='k')
plot = fig_val_acc.add_subplot(111)

plt.title('Validation Accuracy', fontsize=fs_title)


for i in range(1,len(log_dict)):
    try:
        time = log_dict[i]['time']
        acc = log_dict[i]['val_accuracy']
        if acc != None:
            plt.plot(time, acc, marker = 'o', color = 'blue')
    except:
        pass

plt.xlabel('Time [s]', fontsize = fs_ax_labels)
plt.gcf().set_size_inches(15, 6)
plot.tick_params(axis='both', which='major', labelsize=fs_tick_labels)
plot.yaxis.grid()
plt.show()

In [ ]:
### Save graphics
savepath = '../../../data/models/convai2/profilememory/'
fig_trloss.savefig(savepath+'tr_loss.pdf',bbox_inches='tight', format='pdf')
fig_val_acc.savefig(savepath+'val_acc.pdf',bbox_inches='tight', format='pdf')
fig_tr_acc.savefig(savepath+'tr_acc.pdf',bbox_inches='tight', format='pdf')
fig_f1hits.savefig(savepath+'f1_hits.pdf',bbox_inches='tight', format='pdf')
fig_hitsX.savefig(savepath+'Xhits.pdf',bbox_inches='tight', format='pdf')
fig_ppl.savefig(savepath+'PPL.pdf',bbox_inches='tight', format='pdf')



#fig_combi_model.savefig(path+'acc_combi.pdf',bbox_inches='tight', format='pdf')

#fig_overfitting.savefig(path+'overfitting.pdf',bbox_inches='tight', format='pdf')
#fig_3acc.savefig(path+'3_accuracies.pdf',bbox_inches='tight', format='pdf')
#fig_val_matrix.savefig(path+'val_conf_matrix.pdf',bbox_inches='tight', format='pdf')
#fig_test_matrix.savefig(path+'test_conf_matrix.pdf',bbox_inches='tight', format='pdf')